# Semantic Segmentation with on-the-fly Data Augmentation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
import os
from gm_networks import unet_model
from utils import center_crop, normalize_data, create_dataset_from_one_image

In [ ]:
# Load the initial input/output images. Normalize the input images to [0, 1] 
x_initial_train = normalize_data(plt.imread('train/101-INPUT.jpg'))
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = center_crop(x_initial_train, (2560, 2560))
y_initial_train = center_crop(y_initial_train, (2560, 2560))

In [ ]:
# Model/training hyperparameters
INPUT_SHAPE = (256, 256, 3)
INITIAL_FILTERS = 32
BATCH_SIZE = 1
EPOCHS = 200

In [ ]:
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS)

In [ ]:
# Create a dataset with independent patches
MODE = 'independent_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE)

In [ ]:
# Create a dataset with overlapped patches
MODE = 'overlapped_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE, patches_step=256)

In [ ]:
# Create a dataset with random patches
MODE = 'random_patches'
x_train, y_train = create_dataset_from_one_image(x_initial_train, y_initial_train, INPUT_SHAPE[0], mode=MODE, new_images_number=81)

In [6]:
print(x_train.shape)

(100, 256, 256, 3)


In [7]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect') 

# TODO: Post a question in StackOverflow to check why nan values are created, when I use the 'normalize_data' 
# function (from utils.py) as a preprocessing_function in x_datagen_args

y_datagen_args = x_datagen_args.copy()
y_datagen_args['preprocessing_function'] = lambda x: np.where(x>0, 1, 0).astype(x.dtype)

In [8]:
# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [9]:
training_images_number = x_train.shape[0]
steps_per_epoch = training_images_number // BATCH_SIZE

# Define the model and train
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS)

Epoch 1/200


2022-05-24 01:17:50.479842: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


100/100 [==============================] - 4s 10ms/step - loss: 0.7411 - accuracy: 0.5241
Epoch 2/200
100/100 [==============================] - 1s 10ms/step - loss: 0.7007 - accuracy: 0.5643
Epoch 3/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6859 - accuracy: 0.5847
Epoch 4/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6759 - accuracy: 0.5989
Epoch 5/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6718 - accuracy: 0.6039
Epoch 6/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6661 - accuracy: 0.6104
Epoch 7/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6630 - accuracy: 0.6119
Epoch 8/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6578 - accuracy: 0.6180
Epoch 9/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6559 - accuracy: 0.6253
Epoch 10/200
100/100 [==============================] - 1s 10ms/step - loss: 0.6509 - accuracy

100/100 [==============================] - 1s 10ms/step - loss: 0.3760 - accuracy: 0.8506
Epoch 81/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3542 - accuracy: 0.8654
Epoch 82/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3656 - accuracy: 0.8549
Epoch 83/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3727 - accuracy: 0.8502
Epoch 84/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3669 - accuracy: 0.8573
Epoch 85/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3491 - accuracy: 0.8694
Epoch 86/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3482 - accuracy: 0.8640
Epoch 87/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3326 - accuracy: 0.8726
Epoch 88/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3336 - accuracy: 0.8724
Epoch 89/200
100/100 [==============================] - 1s 10ms/step - loss: 0.3224 - 

100/100 [==============================] - 1s 10ms/step - loss: 0.1709 - accuracy: 0.9368
Epoch 160/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1800 - accuracy: 0.9340
Epoch 161/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1515 - accuracy: 0.9457
Epoch 162/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1750 - accuracy: 0.9331
Epoch 163/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1536 - accuracy: 0.9427
Epoch 164/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1618 - accuracy: 0.9416
Epoch 165/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1506 - accuracy: 0.9436
Epoch 166/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1533 - accuracy: 0.9448
Epoch 167/200
100/100 [==============================] - 1s 10ms/step - loss: 0.1373 - accuracy: 0.9507
Epoch 168/200
100/100 [==============================] - 1s 10ms/step - loss: 

In [10]:
# save the model and history of the training (loss, accuracy)
model_path = f'trained_models/cropped_0.001/unet_inputsize{INPUT_SHAPE[0]}_initialfilters{INITIAL_FILTERS}_{MODE}_initialimagesnumber{x_train.shape[0]}_optimizerSGD_lr{"{:0.4f}".format(model.optimizer.lr.value().numpy())}_epochs{EPOCHS}' #TODO: loss and metric'
model.save(model_path)

INFO:tensorflow:Assets written to: trained_models/cropped_0.001/unet_inputsize256_initialfilters32_independent_patches_initialimagesnumber100_optimizerSGD_lr0.0100_epochs200/assets


INFO:tensorflow:Assets written to: trained_models/cropped_0.001/unet_inputsize256_initialfilters32_independent_patches_initialimagesnumber100_optimizerSGD_lr0.0100_epochs200/assets


In [11]:
np.save(os.path.join(model_path, 'history'), history.history)

In [ ]:
type(model.optimizer.lr.value().numpy())

In [ ]:
"{:0.5f}".format(model.optimizer.lr.value().numpy())